In [39]:
from operator import index

import pandas as pd
import folium
import seaborn as sns # pip install seaborn
import matplotlib.pyplot as plt
import plotly.express as px # pip install plotly

from collections import Counter
# from keplergl import KeplerGl

plt.rc('font', family='Malgun Gothic') # 한글 폰트 설정 (Malgun Gothic)

plt.rc('axes', unicode_minus=False) # 마이너스 기호 깨짐 방지

# 공용 변수 선언
dataIn = './../dataIn/'
dataOut = './../dataOut/'

In [40]:
# 이미 생성해 놓은 매장 정보 파일을 읽어 옵니다.
starbucks = pd.read_csv(dataOut + 'starbucks_file.csv')
ediya = pd.read_csv(dataOut + 'ediya_file.csv')
hollysBefore = pd.read_csv(dataOut + 'hollys_file.csv')

In [41]:
print(f'스타벅스 매장 갯수 : {len(starbucks)}')
print(f'이디야 매장 갯수 : {len(ediya)}')
print(f'할리스 매장 갯수 : {len(hollysBefore)}')

스타벅스 매장 갯수 : 666
이디야 매장 갯수 : 474
할리스 매장 갯수 : 122


In [42]:
starbucks.columns

Index(['브랜드', '상호', '주소', '시도', '군구', '위도', '경도'], dtype='object')

In [43]:
ediya.columns

Index(['브랜드', '상호', '주소', '시도', '군구', '위도', '경도'], dtype='object')

In [44]:
hollysBefore.columns

Index(['브랜드', '상호', '주소', '시도', '군구', '위도', '경도', '전화번호', '서비스'], dtype='object')

In [45]:
common_col = starbucks.columns.intersection(hollysBefore.columns)
common_col

Index(['브랜드', '상호', '주소', '시도', '군구', '위도', '경도'], dtype='object')

In [46]:
hollys = hollysBefore[common_col]
hollys.columns

Index(['브랜드', '상호', '주소', '시도', '군구', '위도', '경도'], dtype='object')

In [47]:
print('# 각 매장의 데이터 프레임을 하바로 모아줍니다.')
coffeeFrame = pd.concat([starbucks,ediya,hollys], axis= 0)
print(f'전체 매장 갯수 : {len(coffeeFrame)}')

# 각 매장의 데이터 프레임을 하바로 모아줍니다.
전체 매장 갯수 : 1262


In [48]:
filename = dataOut + 'coffee_list.csv'
coffeeFrame.to_csv(filename, index= False, encoding= 'UTF-8')
print(f'{filename} 파일이 저장되었습니다.')

./../dataOut/coffee_list.csv 파일이 저장되었습니다.


In [49]:
coffeeFrame.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1262 entries, 0 to 121
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   브랜드     1262 non-null   object 
 1   상호      1262 non-null   object 
 2   주소      1262 non-null   object 
 3   시도      1262 non-null   object 
 4   군구      1262 non-null   object 
 5   위도      1260 non-null   float64
 6   경도      1260 non-null   float64
dtypes: float64(2), object(5)
memory usage: 78.9+ KB


In [50]:
# notnull() 함수는 결측지가 아니면 True를 반환해주는 함수 <-> isnull()
print(f'위도/경도 결측지 제거전 : {len(coffeeFrame)}')
mapFrame = coffeeFrame[coffeeFrame['위도'].notnull()]

위도/경도 결측지 제거전 : 1262


In [51]:
print(f'위도/경도 결측지 제거후 : {len(mapFrame)}')

위도/경도 결측지 제거후 : 1260


In [58]:
# 지도 위에 매장 표시
# folium에서 사용할 브랜드별 색상 지정
brand_color = {
    '스타벅스' : 'beige',
    '이디야' : 'darkpurple',
    '할리스' : 'lightblue',
}
seoul_center = [mapFrame['위도'].mean(), mapFrame['경도'].mean()] # 서울 중심 위치
seoul_center

[np.float64(37.541941975611785), np.float64(126.98842106436308)]

In [66]:
# 서울 지도 객체
seoul_map = folium.Map(location= seoul_center, zoom_start=11)
# iterrows() : iterable rows(전체 행을 반복)
# for 색인_번호, 데이터_1행 in mapFrame.iterrows():
# 찾아 볼것 : 마커 크기 조정, popup의 모양
for _, row in mapFrame.iterrows():
    folium.Marker(
        location=[row['위도'],row['경도']],
        popup=f"{row['상호']}({row['브랜드']})", # 팝업 예시 : xxx점(스타벅스)
        icon=folium.Icon(color=brand_color.get(row['브랜드'], 'gray'))
    ).add_to(seoul_map)

htmlfile = dataOut + 'coffee_map.html'
seoul_map.save(htmlfile)
print(f'{htmlfile} 파일 생성')


./../dataOut/coffee_map.html 파일 생성


In [67]:
seoul_map

In [68]:
print('# 군구 컬럼이 마포구, 용산구, 서대문구인 항목들만 추출')
concern_gu = ['마포구', '용산구', '서대문구']
myfilter01 = mapFrame[mapFrame['군구'].isin(concern_gu)]
print(len(myfilter01))

# 군구 컬럼이 마포구, 용산구, 서대문구인 항목들만 추출
153


In [81]:
my_center = [myfilter01['위도'].mean(), myfilter01['경도'].mean()] # 서울 중심 위치
my_center
# 서울 지도 객체
filter_map = folium.Map(location= my_center, zoom_start=13, tiles='OpenStreetMap')
# iterrows() : iterable rows(전체 행을 반복)
# for 색인_번호, 데이터_1행 in myfilter01.iterrows():
# 찾아 볼것 : 마커 크기 조정, popup의 모양
for _, row in myfilter01.iterrows():
    ic_color = 'white'
    folium.Marker(
        location=[row['위도'],row['경도']],
        popup=row['주소'],
        tooltip=f"{row['상호']}({row['브랜드']})", # 팝업 예시 : xxx점(스타벅스)
        icon=folium.Icon(
            color=brand_color.get(row['브랜드'], 'gray'),
            icon_color=ic_color,
            icon='coffee',
            prefix='fa'
                         )
    ).add_to(filter_map)

htmlfile = dataOut + 'coffee_map2.html'
seoul_map.save(htmlfile)
print(f'{htmlfile} 파일 생성')

./../dataOut/coffee_map2.html 파일 생성


In [82]:
filter_map